<a href="https://colab.research.google.com/github/hmatny/question_answering/blob/master/Anserini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup

#### Download and install maven

In [1]:
!wget http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
!tar -xvzf apache-maven-3.6.1-bin.tar.gz -C /opt

--2019-04-24 14:36:02--  http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
Resolving mirror.reverse.net (mirror.reverse.net)... 208.100.14.200
Connecting to mirror.reverse.net (mirror.reverse.net)|208.100.14.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9136463 (8.7M) [application/x-gzip]
Saving to: ‘apache-maven-3.6.1-bin.tar.gz’

apache-maven-3.6.1- 100%[===================>]   8.71M  8.51MB/s    in 1.0s    

2019-04-24 14:36:03 (8.51 MB/s) - ‘apache-maven-3.6.1-bin.tar.gz’ saved [9136463/9136463]

apache-maven-3.6.1/README.txt
apache-maven-3.6.1/LICENSE
apache-maven-3.6.1/NOTICE
apache-maven-3.6.1/lib/
apache-maven-3.6.1/lib/slf4j-api.license
apache-maven-3.6.1/lib/checker-compat-qual.license
apache-maven-3.6.1/lib/jsr250-api.license
apache-maven-3.6.1/lib/jcl-over-slf4j.license
apache-maven-3.6.1/lib/org.eclipse.sisu.plexus.license
apache-maven-3.6.1/lib/animal-sniffer-annotations.license
apache-maven-3.6.

#### Set environment variables

In [2]:
import os
os.environ['PATH'] += ':/usr/lib/jvm/java-8-openjdk-amd64/bin:/opt/apache-maven-3.6.1/bin'
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['M2_HOME'] = "/opt/apache-maven-3.6.0"

# sanity check
!mvn -version

Apache Maven 3.6.1 (d66c9c0b3152b2e69ee9bac180bb8fcc8e6af555; 2019-04-04T19:00:29Z)
Maven home: /opt/apache-maven-3.6.1
Java version: 1.8.0_191, vendor: Oracle Corporation, runtime: /usr/lib/jvm/java-8-openjdk-amd64/jre
Default locale: en_US, platform encoding: UTF-8
OS name: "linux", version: "4.14.79+", arch: "amd64", family: "unix"


#### Clone Anserini git repo

In [3]:
!git clone https://github.com/castorini/Anserini.git
!cd Anserini && mvn clean package appassembler:assemble

Cloning into 'Anserini'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 7035 (delta 33), reused 21 (delta 5), pack-reused 6967
Receiving objects: 100% (7035/7035), 11.17 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (3699/3699), done.
[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.7.0/maven-compiler-plugin-3.7.0.pom (11 kB at 22 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/30/maven-plugins-30.pom (10 kB at 199 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/30/maven-parent-30.pom (41 kB at 843 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/18/apache-18.pom (16 kB at 402 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org

#### Get CAR17 corpus and index data

In [0]:
# !wget http://trec-car.cs.unh.edu/datareleases/v2.0/paragraphCorpus.v2.0.tar.xz
# !tar -xvf paragraphCorpus.v2.0.tar.xz -C /content/Anserini/

In [0]:
# !cd /content/Anserini/ && sh target/appassembler/bin/IndexCollection -collection CarCollection \
# -generator LuceneDocumentGenerator -threads 500000 -input /content/Anserini/paragraphCorpus -index \
# lucene-index.car17a.pos+docvectors+rawdocs -storePositions -storeDocvectors \
# -storeRawDocs

In [0]:
# tar index
# !cd /content/Anserini/ && tar czf lucene-index.car17-large.tar.gz lucene-index.car17a.pos+docvectors+rawdocs

In [0]:
# save index to gs bucket
# !gsutil cp /content/Anserini/lucene-index.car17-large.tar.gz gs://cs378_bert/lucene-index-large.tar.gz

#### Download the Lucene index

In [8]:
!gsutil cp gs://cs378_bert/lucene-index-large.tar.gz /tmp/lucene-index-large.tar.gz
!cd /tmp && tar -xvzf lucene-index-large.tar.gz -C /content/Anserini/

Copying gs://cs378_bert/lucene-index-large.tar.gz...
/ [1 files][ 14.1 GiB/ 14.1 GiB]   83.0 MiB/s                                   
Operation completed over 1 objects/14.1 GiB.                                     
lucene-index.car17a.pos+docvectors+rawdocs/
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tip
lucene-index.car17a.pos+docvectors+rawdocs/_2.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_2.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_3.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_0.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_0_Lucene70_0.dvd
lucene-index.car17a.pos+docvectors+rawdocs/_1.tvx
lucene-index.car17a.pos+docvectors+rawdocs/_0.si
lucene-index.car17a.pos+docvectors+rawdocs/_1.fdx
lucene-index.car17a.pos+d

#### Install dependencies

In [9]:
!pip3 install pyjnius

!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py

  Stored in directory: /root/.cache/pip/wheels/c1/2d/85/9884050da2f10b9f72b029f34bedef0993c339437aa956906f
Successfully built pyjnius
--2019-04-24 14:46:42--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.01s   

2019-04-24 14:46:42 (2.99 MB/s) - ‘modeling.py’ saved [37922/37922]

--2019-04-24 14:46:43--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

### Using Anserini 

In [10]:
# Only run this once
import jnius_config
jnius_config.set_classpath('Anserini/target/anserini-0.4.1-SNAPSHOT-fatjar.jar')

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
questions = ['What is Google?', 'Who is Michael Jordan?']

In [0]:
import run_squad
from nltk.tokenize import sent_tokenize
from jnius import autoclass
JString = autoclass('java.lang.String')
JSearcher = autoclass('io.anserini.search.SimpleSearcher')

searcher = JSearcher(JString('Anserini/lucene-index.car17a.pos+docvectors+rawdocs'))

def get_context(question,filename):
  hits = searcher.search(JString(question),10)
  context_dict = {}
  data = {}
  data["title"] = filename
  counter = 0
  p_list = []
  
  score_map = {}
  for hit in hits:
    p_context = {}
    p_context["context"] = hit.content
    p_context["qas"] = [{"answers": [], "question":question, "id":counter}]
    p_list.append(p_context)
     
    
    counter +=1 
    score_map[counter] = hit.score

  data["paragraphs"] = p_list
  
  root_dict = {}
  root_dict["data"] = [data]
  
  return root_dict,score_map



  
# examples = []
# for question in questions:
#   hits = searcher.search(JString(question),20)
#   for hit in hits:
#     example = run_squad.SquadExample("0", question, sent_tokenize(hit.content))
#     examples.append(example)
    


# BERTsirini - setting up env


Download Required material

In [15]:
#these download all the reqiured BERT material, not sure if they are all required yet
!mkdir /content/BERT/

!gsutil cp -r gs://cs378_bert/model_repo  /content/BERT/
# !gsutil cp -r gs://cs378_bert/bert-tfhub  /content/BERT/  
# !gsutil cp -r gs://cs378_bert/bert-checkpoints  /content/BERT/
 

Copying gs://cs378_bert/model_repo/outputs/checkpoint...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555284422.f53183836553...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555358512.26db6926706b...
\ [3 files][ 20.6 MiB/ 20.6 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555359174.26db6926706b...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555359398.26db6926706b...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555516188.fa30e7de8aaf...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555707495.dd846ac09b98...
Copying gs://cs378_bert/model_repo/outputs/graph.pbtxt...
Copying gs:/

TPU settings

In [0]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import zipfile
from matplotlib import pyplot as plt
%matplotlib inline
import sys
import datetime
import json
import tensorflow as tf

In [17]:
#NOT SURE IF WE NEED THIS RIGHT NOW

# BUCKET = 'cs378_bert' #@param {type:"string"}
# assert BUCKET, 'Must specify an existing GCS bucket name'
# REPO ='model_repo'
# BUCKET_OUTPUT_DIR = 'gs://{}/{}/outputs'.format(BUCKET,REPO)
# BUCKET_PREFIX = "gs://cs378_bert"
# tf.gfile.MakeDirs(BUCKET_OUTPUT_DIR)
# print('***** Model output directory: {} *****'.format(BUCKET_OUTPUT_DIR))
 
# !wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py
!wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 

--2019-04-24 14:49:14--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6258 (6.1K) [text/plain]
Saving to: ‘optimization.py.1’

optimization.py.1   100%[===================>]   6.11K  --.-KB/s    in 0s      

2019-04-24 14:49:14 (118 MB/s) - ‘optimization.py.1’ saved [6258/6258]

--2019-04-24 14:49:15--  https://raw.githubusercontent.com/google-research/bert/master/tokenization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12257 (12K) [text/plain]
Saving to

In [26]:
import json
# grab context from anserini
# format context+question in json for bert to read
QUESTION = 'Why did Mark Twain call the 19th century the gilded age?' #@param {type:"string"}
FILENAME = 'gilded_age' #@param {type:"string"}
context_dict,scores = get_context(QUESTION,FILENAME)
FILENAME = FILENAME + ".json"
text_file = open(FILENAME, "w")
text_file.write(json.dumps(context_dict))
# pass input file to bert 
### TYPE FILENAME INTO predict-file= 
!python run_squad.py \
  --vocab_file='BERT/model_repo/uncased_L-12_H-768_A-12/vocab.txt' \
  --bert_config_file='BERT/model_repo/uncased_L-12_H-768_A-12/bert_config.json' \
  --init_checkpoint='BERT/model_repo/outputs/' \
  --do_predict=True \
  --predict_file=$FILENAME \
  --train_batch_size=1 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir='./' > output.txt
bert_pred = open("nbest_predictions.json")
bert_pred = json.load(bert_pred)

INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf09405eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_

In [0]:
# !cat predictions.json
# !cat nbest_predictions.json
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
def calc_score(a_score, b_score,u=0.5):
  return (1-u)*a_score + u*b_score

def normalize(d, target=1.0):
   raw = sum(d.values())
   factor = target/raw
   return {key:value*factor for key,value in d.items()}

def find_best_ans(bert_pred,a_scores,u=0.5):
  u = u
  a_scores = normalize(a_scores,1) #normalize values in anserini scores

  best = 0
  max_score = -1
  for key in range(0,len(bert_pred.keys())-1):
    ans_ind = key+1 #anserini keys indexed at 1
    score_temp = calc_score(a_scores[ans_ind],bert_pred[str(key)][0]["probability"],u)
    if score_temp > max_score:
        max_score = score_temp
        best_key = key
  return best_key, max_score

def print_results(QUESTION, b_key,m_score):
  answer = bert_pred[str(b_key)][0]["text"]
  context = context_dict["data"][0]["paragraphs"][b_key]["context"]
  
  print("~~~~~~~~~~~~~~~~~~~~")
  print("Confidence score: " + str(m_score)+ "\n")
  print("Question: " + QUESTION+ "\n")
  print("Answer: " + bert_pred[str(b_key)][0]["text"] + "\n")
  print("Context found in:" + "\n")
  pp.pprint(context)
  print("~~~~~~~~~~~~~~~~~~~~")
  


In [49]:
#open nbest_predictions
b_key,m_score = find_best_ans(bert_pred, scores, 0.5)
print_results(QUESTION, b_key, m_score)

~~~~~~~~~~~~~~~~~~~~
Confidence score: 0.4775143259677459

Question: Why did Mark Twain call the 19th century the gilded age?

Answer: there had been a dramatic expansion of American wealth and prosperity

Context found in:

('The "Gilded Age" was a term that Mark Twain used to describe the period of '
 'the late 19th century when there had been a dramatic expansion of American '
 'wealth and prosperity. Reform of the Age included the Civil Service Act, '
 'which mandated a competitive examination for applicants for government jobs. '
 'Other important legislation included the Interstate Commerce Act, which '
 "ended railroads' discrimination against small shippers, and the Sherman "
 'Antitrust Act, which outlawed monopolies in business. Twain believed that '
 'this age was corrupted by such elements as land speculators, scandalous '
 'politics, and unethical business practices. Since the days of Charles A. '
 'Beard and Matthew Josephson, some historians have argued that the United '